<h2><u>Situation</u></h2>
The data has been gathered! However, the data is not perfect. , some numbers that are important have been dropped by accident!

In [357]:
import pandas as pd
DATA = pd.read_csv("MovieDB.csv")

Some websites have misspelled movie titles

In [364]:
DATA[DATA.Title.str.startswith("Marg")][-3:-1].Title

1070    Margaret
1218    Margeret
Name: Title, dtype: object

Some movies have the numbers dropped off by accident while scraping. Toy Story needs to be either Toy Story 3 or Toy Story 4! 12 Years a Slave is missing the 12!

In [366]:
DATA[DATA.Title.str.startswith("Toy")][-3:-1] 

,Title,Website,Rank
1015,Toy Story,Columbus Underground,2
1028,Toy Story,Columbus Underground,15


In [371]:
DATA[DATA.Title.str.startswith("Years")]

,Title,Website,Rank
972,Years a Slave,CNET,23
1074,Years a Slave,Town and Country Mag,-1


Some are mis-titled! It is not Art of Killing! It's Act of Killing.

In [372]:
DATA[DATA.Title.str.endswith("Killing")]

,Title,Website,Rank
37,The Act of Killing,Vulture,38
78,The Art of Killing,No Film School,-1
376,The Act Of Killing,AVClub,15
1249,The Act of Killing,Irish Times,8
1354,The Act of Killing,The Playlist,3


In [374]:
len(set(DATA.Title))

761

<h2><u> Task </u></h2>
There are 761 unique titles in my database. However several are misspelt or misaligned. I need to fix this be left with a list of titles which are easy to process and don't contain any spurious duplicates. This task is called fuzzy de-duplication.

<h2><u> Action </u></h2>
I did the fuzzy de-duplication using two techniques:

1. Shingling to identify fuzzy matches.

2. Entity resoltion using the dedupe package to automate the de-duplication process!

This is the code used for shingling.

In [375]:
#Source for this code: https://mattilyra.github.io/2017/05/23/document-deduplication-with-lsh.html

import itertools
# from lsh import lsh, minhash # https://github.com/mattilyra/lsh

# a pure python shingling function that will be used in comparing
# LSH to true Jaccard similarities
def get_shingles(text, char_ngram=2):
    """Create a set of overlapping character n-grams.
    
    Only full length character n-grams are created, that is the first character
    n-gram is the first `char_ngram` characters from text, no padding is applied.

    Each n-gram is spaced exactly one character apart.

    Parameters
    ----------

    text: str
        The string from which the character n-grams are created.

    char_ngram: int (default 5)
        Length of each character n-gram.
    """
    return set(text[head:head + char_ngram] for head in range(0, len(text) - char_ngram))


def jaccard(set_a, set_b):
    """Jaccard similarity of two sets.
    
    The Jaccard similarity is defined as the size of the intersection divided by
    the size of the union of the two sets.

    Parameters
    ---------
    set_a: set
        Set of arbitrary objects.

    set_b: set
        Set of arbitrary objects.
    """
    intersection = set_a & set_b
    union = set_a | set_b
    if len(union) != 0 :
        return len(intersection) / len(union)


In [278]:
#initially jaccard was returning None - found out it was becaue of short titles so I changed the 
#parameter ngram to 3
DATA['Title'][1001]

'Us'

This is a function that prints out all the duplicates it finds. It has a parameter threshold which I will use to keep fine tuning my results.

In [279]:
def find_duplicates(data, threshold = 0.9):
    shingles = []
    for text in data:
        shingles.append(get_shingles(text.lower()))
    shingles
    duplicates = []
    drop_rows = []
    for i_doc in range(len(shingles)):
        for j_doc in range(i_doc + 1, len(shingles)):
            jaccard_similarity = jaccard(shingles[i_doc], shingles[j_doc])
            if jaccard_similarity == None: #for titles with length smaller than 3
                continue
            else: is_duplicate = 0.99 > jaccard_similarity >= threshold
            if is_duplicate:
                duplicates.append((i_doc, j_doc, jaccard_similarity))
    for i, j, score in duplicates:
        print(data[i]+  " , " + data[j] + str(score))
    return duplicates

print(len(find_duplicates(DATA['Title'])))

Mad Max: Fury Road , Mad Max: Fury Road   0.9375
The Tree of Life , The Tree of Life  0.9285714285714286
Spider-Man: Into the Spider-Verse , Spider-Man: Into the Spider-Verse  0.96
Spider-Man: Into the Spider-Verse , Spider-Man: Into the Spider-Verse   0.9230769230769231
Spider-Man: Into the Spider-Verse , Spider-Man: Into The Spiderverse0.92
Once Upon a Time in Hollywood , Once Upon A Time… In Hollywood0.9230769230769231
You Were Never Really Here , You Were Never Really Here   0.9545454545454546
The Social Network , The Social Network 0.9411764705882353
Once Upon a Time in Anatolia , Once Upon A Time In Anatolia    0.9166666666666666
Two Days One Night , Two Days  One Night0.9411764705882353
The Favourite , The Favourite  0.9166666666666666
The Tree of Life , The Tree of Life  0.9285714285714286
Mad Max: Fury Road , Mad Max: Fury Road   0.9375
The Wolf of Wall Street , The Wolf of Wall Street  0.9047619047619048
The Social Network , The Social Network 0.9411764705882353
Interstellar 

I found 173 duplicates where the main problem was spaces, so the first step is to just remove the trailing spaces using strip.

In [280]:
DATA['Title'] = DATA['Title'].str.strip()

In [281]:
len(find_duplicates(DATA.Title))

Spider-Man: Into the Spider-Verse , Spider-Man: Into The Spiderverse0.92
Once Upon a Time in Hollywood , Once Upon a Time   in Hollywood0.96
Once Upon a Time in Hollywood , Once Upon A Time… In Hollywood0.9230769230769231
Two Days One Night , Two Days  One Night0.9411764705882353
Upstream Color , Upstream Colour0.9230769230769231
Once Upon a Time   in Hollywood , Once Upon a Time in Hollywood0.96
Once Upon a Time   in Hollywood , Once Upon a Time In Hollywood0.96
Spider-Man: Into the Spider-Verse , Spider-Man: Into The Spiderverse0.92
Once Upon a Time in Hollywood , Once Upon A Time… In Hollywood0.9230769230769231
Mission: Impossible – Fallout , Mission: Impossible Fallout0.92
Spider-Man: Into the Spider-Verse , Spider-Man: Into The Spiderverse0.92
Spider-Man: Into The Spiderverse , Spider-Man: Into the Spider-Verse0.92
Spider-Man: Into The Spiderverse , Spider-Man: Into the Spider-Verse0.92
Spider-Man: Into The Spiderverse , Spider-Man: Into the Spider-Verse0.92
Spider-Man: Into The S

22

Only 22 duplicates now with high threshold. Once I changed the threshold I understood how messy the data really was!

In [283]:
len(find_duplicates(DATA.Title, 0.7))

Mad Max: Fury Road , Mad Max: Fury Road (2015)0.75
Mad Max: Fury Road , Mad Max: Fury Road (2015)0.75
Mad Max: Fury Road , Mad Max: Fury Road (2015)0.75
Mad Max: Fury Road , Mad Max: Fury Road (2015)0.75
The Tree of Life , Tree of Life0.7692307692307693
The Tree of Life , Tree Of Life0.7692307692307693
Spider-Man: Into the Spider-Verse , Spider-Man: Into The Spiderverse0.92
Spider-Man: Into the Spider-Verse , SpiderMan: Into The SpiderVerse0.8076923076923077
Spider-Man: Into the Spider-Verse , SpiderMan: Into the SpiderVerse0.8076923076923077
Spider-Man: Into the Spider-Verse , SpiderMan: Into the SpiderVerse0.8076923076923077
The Florida Project , The Florida Project (2017)0.7083333333333334
Once Upon a Time in Hollywood , Once Upon a Time   in Hollywood0.96
Once Upon a Time in Hollywood , Once Upon A Time… In Hollywood0.9230769230769231
Once Upon a Time in Hollywood , Once Upon a Time...in Hollywood0.8518518518518519
Once Upon a Time in Hollywood , Once Upon a Time in Hollywood (2019

404

Oh no, while scraping I messed up and removed the numbers on Toy Story!

In [284]:
DATA.loc[DATA['Title'].str.startswith("Toy", na=False)]

,Title,Website,Rank
113,Toy Story 3,Insider,78
238,Toy Story 3,GamesRadar,53
494,Toy Story 3 (2010),The Stacker,35
741,Toy Story 3,Rotten Tomatoes,-1
742,Toy Story 4,Rotten Tomatoes,-1
969,Toy Story,CNET,20
981,Toy Story,Looper,-1
1015,Toy Story,Columbus Underground,2
1028,Toy Story,Columbus Underground,15
1143,Toy Story 3,IMDB User,9


I re-verified the sources and updated the data later w/

CNET: "Toy Story 3"

Columbus Rank 2: "Toy Story 3"

Columbus Rank 15: "Toy Story 4"

Looper: "Toy Story 3"

I noticed that Paddington also had similar issues, but upon examining the data I found out that it was accurate and non-duplicate

In [286]:
DATA.loc[DATA['Title'].str.startswith("Paddington", na=False)] 

,Title,Website,Rank
59,Paddington 2,No Film School,-1
165,Paddington 2,Insider,26
207,Paddington 2,GamesRadar,84
314,Paddington 2,AVClub,78
392,Paddington 2,Esquire,10
692,Paddington,Rotten Tomatoes,-1
693,Paddington 2,Rotten Tomatoes,-1
916,Paddington 2,Independent,7


In [287]:
len(find_duplicates(DATA.Title, 0.7))

Mad Max: Fury Road , Mad Max: Fury Road (2015)0.75
Mad Max: Fury Road , Mad Max: Fury Road (2015)0.75
Mad Max: Fury Road , Mad Max: Fury Road (2015)0.75
Mad Max: Fury Road , Mad Max: Fury Road (2015)0.75
The Tree of Life , Tree of Life0.7692307692307693
The Tree of Life , Tree Of Life0.7692307692307693
Spider-Man: Into the Spider-Verse , Spider-Man: Into The Spiderverse0.92
Spider-Man: Into the Spider-Verse , SpiderMan: Into The SpiderVerse0.8076923076923077
Spider-Man: Into the Spider-Verse , SpiderMan: Into the SpiderVerse0.8076923076923077
Spider-Man: Into the Spider-Verse , SpiderMan: Into the SpiderVerse0.8076923076923077
The Florida Project , The Florida Project (2017)0.7083333333333334
Once Upon a Time in Hollywood , Once Upon a Time   in Hollywood0.96
Once Upon a Time in Hollywood , Once Upon A Time… In Hollywood0.9230769230769231
Once Upon a Time in Hollywood , Once Upon a Time...in Hollywood0.8518518518518519
Once Upon a Time in Hollywood , Once Upon a Time in Hollywood (2019

404

Okay, a bunch of the movies have the date left in them. I need to fix this. Easy enough.

In [288]:
DATA.loc[DATA['Title'].str.endswith(')')]

,Title,Website,Rank
429,Manuscripts Don t Burn (2013),The Stacker,100
430,The Arbor (2010),The Stacker,99
431,A Film Unfinished (2010),The Stacker,98
432,Selma (2014),The Stacker,97
433,Blue Is the Warmest Color (2013),The Stacker,96
434,The King s Speech (2010),The Stacker,95
435,Inside Job (2010),The Stacker,94
436,Western (2015),The Stacker,93
437,Zama (2017),The Stacker,92
438,Elle (2016),The Stacker,91


In [289]:
#remove dates in brackets
DATA.Title.replace(regex=r'[\(\[].*?[\)\]]', value='', inplace = True)

Now with all dates removed, I ran the duplicates function!

In [291]:
find_duplicates(DATA.Title, 0.7)

The Tree of Life , Tree of Life0.7692307692307693
The Tree of Life , Tree Of Life0.7692307692307693
The Rider , The Rider 0.875
A Separation , A Separation 0.9090909090909091
Moonlight , Moonlight 0.875
Moonlight , Moonlight 0.875
Moonlight , Moonlight 0.875
The Fits , The Fits 0.8571428571428571
Spider-Man: Into the Spider-Verse , Spider-Man: Into The Spiderverse0.92
Spider-Man: Into the Spider-Verse , SpiderMan: Into The SpiderVerse0.8076923076923077
Spider-Man: Into the Spider-Verse , SpiderMan: Into the SpiderVerse0.8076923076923077
Spider-Man: Into the Spider-Verse , SpiderMan: Into the SpiderVerse0.8076923076923077
The Florida Project , The Florida Project 0.9444444444444444
It’s Such a Beautiful Day , It s Such a Beautiful Day 0.84
Parasite , Parasite 0.8571428571428571
Parasite , Parasite 0.8571428571428571
The Handmaiden , The Handmaiden 0.9230769230769231
The Handmaiden , The Handmaiden 0.9230769230769231
Once Upon a Time in Hollywood , Once Upon a Time   in Hollywood0.96
Onc

Selma  , Selma0.75
Selma  , Selma0.75
Selma  , Selma0.75
Blue Is the Warmest Color  , Blue Is the Warmest Colour0.9166666666666666
The King s Speech  , The King s Speech0.9333333333333333
The King s Speech  , The Kings Speech0.8125
The King s Speech  , The King s Speech0.9333333333333333
Columbus  , Columbus0.8571428571428571
Phoenix  , Phoenix0.8333333333333334
Phoenix  , Phoenix0.8333333333333334
Phoenix  , Phoenix0.8333333333333334
Two Days One Night  , Two Days One Night0.9411764705882353
Two Days One Night  , Two Days One Night0.9411764705882353
Eighth Grade  , Eighth Grade0.9090909090909091
The Artist  , The Artist0.8888888888888888
The Farewell  , The Farewell0.9090909090909091
The Tale of The Princess Kaguya  , The Tale of the Princess Kaguya0.96
The Tale of The Princess Kaguya  , The Tale of the Princess Kaguya0.96
American Hustle  , American Hustle0.9285714285714286
Paterson  , Paterson0.8571428571428571
Paterson  , Paterson0.8571428571428571
Paterson  , Paterson0.85714285714

[(2, 1059, 0.7692307692307693),
 (2, 1352, 0.7692307692307693),
 (3, 488, 0.875),
 (4, 520, 0.9090909090909091),
 (5, 527, 0.875),
 (5, 548, 0.875),
 (5, 813, 0.875),
 (6, 452, 0.8571428571428571),
 (8, 327, 0.92),
 (8, 900, 0.8076923076923077),
 (8, 951, 0.8076923076923077),
 (8, 999, 0.8076923076923077),
 (9, 490, 0.9444444444444444),
 (11, 473, 0.84),
 (13, 521, 0.8571428571428571),
 (13, 695, 0.8571428571428571),
 (15, 632, 0.9230769230769231),
 (15, 799, 0.9230769230769231),
 (17, 147, 0.96),
 (17, 352, 0.9230769230769231),
 (17, 396, 0.8518518518518519),
 (17, 547, 0.96),
 (17, 801, 0.96),
 (17, 949, 0.96),
 (17, 1009, 0.96),
 (20, 1105, 0.75),
 (21, 513, 0.8333333333333334),
 (25, 518, 0.9411764705882353),
 (25, 541, 0.9411764705882353),
 (25, 796, 0.9411764705882353),
 (25, 815, 0.9411764705882353),
 (26, 487, 0.8571428571428571),
 (27, 462, 0.9333333333333333),
 (32, 443, 0.8333333333333334),
 (34, 366, 0.9411764705882353),
 (34, 444, 0.9411764705882353),
 (35, 466, 0.91666666

Omg. There are way too many titles that have tiny errors. I need a package to fix this! I used the great dedupe function to do this. Uncomment this block to try it on your own!

In [350]:
#!pip install pandas-dedupe
#import pandas_dedupe
#df_final = pandas_dedupe.dedupe_dataframe(DATA,['Title'])
#df_final.to_csv('dedupe.csv')

After training the dedupe package, I manually fixed any other incorrect titles and this is what I was left with!

In [351]:
final = pd.read_csv('final.csv', index_col = 'Index')

In [354]:
find_duplicates(final.Title, 0.6)

paddington 2 , paddington0.8
inside out , inside job0.6
the big short , the big sick0.6153846153846154
inside out , inside job0.6
paddington 2 , paddington0.8
paddington 2 , paddington0.8
inside out , inside job0.6
paddington 2 , paddington0.8
inside out , inside job0.6
paddington 2 , paddington0.8
inside job , inside out0.6
inside job , inside out0.6
inside job , inside out0.6
inside job , inside out0.6
inside job , inside out0.6
the big sick , the big short0.6153846153846154
dawn of the planet of the apes , war for the planet of the apes0.6296296296296297
paddington , paddington 20.8
paddington , paddington 20.8
war for the planet of the apes , dawn of the planet of the apes0.6296296296296297


[(59, 692, 0.8),
 (74, 435, 0.6),
 (135, 580, 0.6153846153846154),
 (148, 435, 0.6),
 (165, 692, 0.8),
 (207, 692, 0.8),
 (283, 435, 0.6),
 (314, 692, 0.8),
 (330, 435, 0.6),
 (392, 692, 0.8),
 (435, 515, 0.6),
 (435, 648, 0.6),
 (435, 946, 0.6),
 (435, 990, 0.6),
 (435, 1188, 0.6),
 (580, 1171, 0.6153846153846154),
 (604, 747, 0.6296296296296297),
 (692, 693, 0.8),
 (692, 916, 0.8),
 (747, 1199, 0.6296296296296297)]

<h2><u> Result </u></h2>

All entities have been resolved! The dataset now has 427 unique titles and I'm fairly certain the entities have all been resolved. Over the course of data analysis and data visualization, if any problems arise I'll fix them on the fly.

The result was I went from 761 non-unique fuzzy titles to 427 unique titles. All the titles have been carefully renamed to be exactly how they are found on IMDB.

In [356]:
print("There are " + str(len(set(dedupe['Title']))) + " unique titles in my dataset")

There are 427 unique titles in my dataset
